In [217]:
# ! pip install spacy
# ! pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.1.0/en_core_web_trf-3.1.0.tar.gz
# ! pip install presidio-analyzer
# ! python -m spacy download en_core_web_trf
# ! python -m spacy download en_core_web_lg

In [218]:
import spacy
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
import pandas as pd

In [219]:
df = pd.read_csv('../../Data/Raw/YelpFakeReview(Tagged DF).csv')
df = df.dropna(subset=['reviewContent'])

### Entity Recognition Extraction from Review using Microsoft Presidio and Spacy model

Presidio Analyzer supports a wide range of entities. Here are some of them that we will be selecting:

- `DATE_TIME`: Absolute or relative dates or periods or times smaller than a day¹.
- `EMAIL_ADDRESS`: An email address identifies an email box to which email messages are delivered¹.
- `NRP`: A person’s Nationality, religious or political group¹.
- `LOCATION`: Name of politically or geographically defined location (cities, provinces, countries, international regions, bodies of water, mountains)¹.
- `PERSON`: A full person name, which can include first names, middle names or initials, and last names¹.
- `PHONE_NUMBER`: A telephone number¹.

In [220]:
# Initialize the Presidio analyzer engine
analyzer = AnalyzerEngine()

In [221]:
# Load the installed model
nlp = spacy.load('en_core_web_trf')

In [222]:
# Create configuration containing engine name and models
configuration = {
    "nlp_engine_name": "spacy",
    "models": [
        {"lang_code": "en", "model_name": "en_core_web_trf"},
    ],
}

# Create NLP engine based on configuration
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine = provider.create_engine()

# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(nlp_engine=nlp_engine, supported_languages=["en"])

In [223]:
df.reviewContent.isna().sum()

0

In [224]:
# Assuming df is your DataFrame and 'reviewContent' is your column with text data
entities = ['DATE_TIME', 'EMAIL_ADDRESS', 'NRP', 'LOCATION', 'PERSON', 'PHONE_NUMBER']

for entity in entities:
    df[entity] = None

In [225]:
# for index, row in df.iterrows():
#     text = row['reviewContent']
#     results = analyzer.analyze(text=text, entities=entities, language='en')
#     for result in results:
#         entity_text = text[result.start : result.end]
#         entity_type = result.entity_type
#         if df.loc[index, entity_type] == None:
#             df.loc[index, entity_type] = [entity_text]
#         else:
#             df.loc[index, entity_type].append(entity_text)

In [226]:
def analyze_row(row):
    text = row['reviewContent']
    results = analyzer.analyze(text=text, entities=entities, language='en')
    for result in results:
        entity_text = text[result.start : result.end]
        entity_type = result.entity_type
        if row[entity_type] == None:
            row[entity_type] = [entity_text]
        else:
            row[entity_type].append(entity_text)
    return row

In [ ]:
df = df.apply(analyze_row, axis=1)

In [307]:
df.head(5)

,reviewDate,reviewID,reviewerID,reviewContent,reviewRating,reviewUsefulCount,reviewCoolCount,reviewFunnyCount,restaurantID,flagged,...,resLocation,resName,resReviewCount,resRating,DATE_TIME,EMAIL_ADDRESS,NRP,LOCATION,PERSON,PHONE_NUMBER
0,9/22/2012,GtwU21YOQn-wf4vWRUIx6w,bNYesZ944s6IJVowOnB0iA,"Unlike Next, which we'd eaten at the previous ...",5,0,0,0,pbEiXam9YJL3neCYHGwLUA,N,...,"Alinea - Lincoln Park - Chicago, IL",Alinea,841,4.5,"['the previous night', 'four hours and thirty-...",NaN,['English'],NaN,"['Willy Wonka', 'Mickey', 'Minnie Mouse', 'Jac...",NaN
1,9/22/2012,0LpVTc3,TRKxLC3y-ZvP45e5iilMtw,Probably one of the best meals I've had ever. ...,5,0,0,0,pbEiXam9YJL3neCYHGwLUA,N,...,"Alinea - Lincoln Park - Chicago, IL",Alinea,841,4.5,"['an evening', 'a least 4 hours']",NaN,NaN,NaN,['Grant Achatz'],NaN
2,9/19/2012,tljtLzf68Fkwf,0EMm8umAqXZzyhxNpL4M9g,Service was impeccable. Experience and present...,3,2,0,0,pbEiXam9YJL3neCYHGwLUA,N,...,"Alinea - Lincoln Park - Chicago, IL",Alinea,841,4.5,NaN,NaN,NaN,NaN,NaN,NaN
3,9/6/2012,iSN,DlwexC7z88ymAzu45skODw,"The problem with places like this, given the e...",3,8,0,3,pbEiXam9YJL3neCYHGwLUA,N,...,"Alinea - Lincoln Park - Chicago, IL",Alinea,841,4.5,['5 seconds'],NaN,NaN,NaN,"['Jackson Pollock', 'Grant Achatz']",NaN
4,9/9/2012,Jmwrh7,kW2dk1CWihmh3g7k9N2G8A,I have no idea how to write my review - dining...,5,1,2,0,pbEiXam9YJL3neCYHGwLUA,N,...,"Alinea - Lincoln Park - Chicago, IL",Alinea,841,4.5,['a few days'],NaN,['American'],['Chicago'],NaN,NaN


In [230]:
df.to_csv('Entity_dataframe.csv', index= False)

In [ ]:
df = pd.read_csv('Entity_dataframe.csv')

In [297]:
import numpy as np
import ast

for entity in entities:
    # Convert the string representation of list to actual list
    df[entity] = df[entity].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.NaN)
    

In [303]:
for entity in entities:
    # Count of elements in the list in each row
    df[entity + '_count'] = df[entity].apply(lambda x: len(x) if isinstance(x, list) else 0)
    df[entity + '_flag'] = df[entity].apply(lambda x: isinstance(x, list))


In [305]:
df.to_csv('Entity_Count_Flag_dataframe.csv',index=None)